TODO
- Normal working hours
- Efficiency within those hours (either in meetings or build work)
- Split between meetings and build
- D3 visualization: https://observablehq.com/@d3/calendar-view

In [ ]:
import requests
from ics import Calendar
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
url = 'https://calendar.google.com/calendar/ical/4kaefeajbrp4ucss3lunngrcq4%40group.calendar.google.com/public/basic.ics'
raw = requests.get(url).text
c = Calendar(raw) #.replace('BEGIN:VCALENDAR', 'BEGIN:VCALENDAR\r\nPRODID:noah-rocks'))
c

In [ ]:
items = []
for e in list(c.events):
    items.append({
        'event_name': e.name,
        'begin': str(e.begin),
        'end': str(e.end),
        'duration_min': (e.duration.total_seconds())/60.0,
        'duration_hr': (e.duration.total_seconds())/60.0/60.0
    })

df = pd.DataFrame(items)
df['begin'] = pd.to_datetime(df.begin)
df['begin'] = df.begin.apply(lambda x: x.replace(tzinfo=None))
df['date'] = df.begin.dt.date
df['end'] = pd.to_datetime(df.end)
df['end'] = df.begin.apply(lambda x: x.replace(tzinfo=None))
df.head()

In [ ]:
df.sort_values(by='date', ascending=False)

In [ ]:
import re

def flip_category(s):
    return ' '.join(s.split(' ')[-1:] + s.split(' ')[:-1])

def sanitize_name(s):
    s_lower = s.lower()
    if ('forward' in s_lower) & ('build' in s_lower):
        return 'Forward: Build'
    if ('forward' in s_lower) & ('meeting' in s_lower):
        return 'Forward: Meeting'
    return s

def extract_people_names(s):
    split_by_w_list = s.split('w/ ')
    
    if len(split_by_w_list) < 2:
        return "SOLO"
    
    name_and_location = split_by_w_list[-1]
    split_by_at_list = name_and_location.split('@')
    
    return name_and_location.strip() if len(split_by_at_list) < 2 else split_by_at_list[0].strip()

def extract_location_with_person(s):
    split_by_w_list = s.split('w/ ')
    
    if len(split_by_w_list) < 2:
        return None
    
    split_by_at_list = split_by_w_list[-1].split('@')
    
    if len(split_by_at_list) < 2:
        return None
    
    return split_by_at_list[-1].strip()
    
    
    

# def extract_names_and_location(s):
#     name_search_matches = re.search(r'w\/\s(.*)\s@?', s)
#     return name_search_matches.group(0)


df['people'] = df['event_name'].apply(extract_people_names)
df['location'] = df['event_name'].apply(extract_location_with_person)
df['activity'] = df['event_name'].apply(lambda x: x.split('w/ ')[0])
df['location_and_activity'] = df.apply(lambda x: '{} @ {}'.format(x['activity'].strip(), x['location'].strip()) if x['location'] is not None else x['activity'].strip(), axis=1)
df['date_m'] = df.begin.dt.strftime('%Y-%m')
df['date_y'] = df['begin'].dt.year
df.head()

#extract_names("Lunch w/ Mike @ Thanh Tham vietnamese")

In [ ]:
#df.location_and_activity.value_counts()
df.people.value_counts()

In [ ]:
for r in df['event_name'].value_counts().iteritems():
    print(r)

In [ ]:
idx = 2

for p in df.people.value_counts().index[1:]:
    dft = df[df['people'] == p].copy()
    dft = dft['location_and_activity'].value_counts()
    print('ACTIVITIES WITH {}'.format(p))
    for r in dft[:10].iteritems():
        print('    {} ({}x)'.format(r[0], r[1]))
    print('   ')


In [ ]:
# df.dtypes
top_n = 20

top_names = list(df.name_extracted.value_counts()[:top_n].index)
# random.shuffle(top_names)
top_names.sort()

dft = df[
    (df['name_extracted'] != 'SOLO') &\
    (df['name_extracted'].isin(top_names))
].copy()
dft = dft.groupby(['date_y', 'name_extracted'])['name'].agg(['count']).reset_index()

# dft is throwaway within a cell
sns.barplot(x='date_y', y='count', hue='name_extracted', data=dft, hue_order=top_names);

In [ ]:
import matplotlib.dates as mdates

In [ ]:
def plot_within_range(df_in, start, end=None):
    if end is None:
        end = '2021-01-01'
    df1 = df_in[
        (df_in['date'] >= pd.to_datetime(start)) &\
        (df_in['date'] < pd.to_datetime(end))
    ].copy()
    dft = df1.groupby(['date', 'name'])['duration_hr'].agg(['sum']).reset_index()
    dft.head()

    my_dates = pd.date_range(dft.date.min(), dft.date.max(), freq='D')
    idx = pd.MultiIndex.from_product([my_dates, dft.name.unique()], names=['date', 'name'])
    dft = dft.set_index(['date', 'name']).reindex(idx).fillna(0).reset_index()
    dft.sort_values(by='date', ascending=False)
    dft['date'] = dft['date'].apply(lambda x: x.strftime('%m/%d/%Y'))

    dft.pivot(index='date', columns='name', values='sum').plot.bar(stacked=True);
    plt.xlabel('');
    plt.ylabel('Productive Hours');

    ax = plt.gca()
    handles, labels = ax.get_legend_handles_labels()
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0], reverse=True))
    ax.legend(handles, labels)
    plt.show()
    return

# For grid view https://observablehq.com/@d3/calendar-view
plt.rcParams['figure.figsize'] = [12, 8]
plot_within_range(df, '2017-04-17', '2020-05-17')

In [ ]:
plt.rcParams['figure.figsize'] = [10, 20]
sns.barplot(y=df.name.value_counts().index, x=df.name.value_counts());